#### Aplicamos algunas transformaciones a los datos de cada plataforma de manera que los datos sean consistentes a la hora de generar con ellos un modelo de aprendizaje 


In [1]:
import pandas as pd

import numpy as np

#funcion para obtener los datasets de google drive
from  raw_data.conexiones_lake import obtener_datos_plataforma

Generamos campo id con prefijo dado por nombre de plataforma

In [2]:
def generar_id(prefijo,df):
  """
  Concatena el valor de prefijo a la columna show_id del df.

  Parameters
  ---------
  prefijo: str
    letra que se añade al inicio de cada valor del campo show_id
  df: object
    dataframe con la columna show_id
  """

  df["id"] = prefijo+df['show_id']  
  
  # eliminamos columna show_id pues no es necesaria con el nuevo id
  df.drop(columns='show_id',index=1,inplace=True)

Reemplazamos valores nulos de rating

In [3]:
def completar_rating(df):
  """
  Completa los nulos del campo rating con letra G.

  Parameters
  ---------
   df: object
    dataframe con la columna rating
  """

  df['rating'].fillna('G',inplace=True)

Formateamos fechas

In [4]:
def formatear_fecha(df):
  """
  Formatea fechas a formato AAA-mm-dd del df.

  Parameters
  ----------
  df: object
    dataframe con la columna que represente fechas
  """

  df['date_added'] = pd.to_datetime(df.date_added)

Campos de texto a minusculas

In [5]:
def texto_en_minuscula(df):
  """
  Convierte todos los campos del df de texto en minúsculas.

  Parameters
  ---------
  df: object
    dataframe con columnas de texto
  """

  # seleccionamos las columnas de texto
  columns = np.array( df.columns)

  # excluimos las de fechas
  columns = [column for column in columns if column != 'date_added' and column != 'release_year']
  
  #solo las seleccionadas van a lowercase
  df[columns] = df[columns].apply(lambda val: val if not np.all(pd.notna(val)) else  val.str.lower())  

Separamos campo duration en duration_int y duration_type

In [6]:
def obtener_valor_tipo(item,posicion):
   
  if isinstance(item,str):
  
    if ' ' in item:
  
      num = item.split()[posicion]            
  
      return num.strip()
  
  return item    

In [19]:
def separar_duration(df):
  """
  Separa el campo duration en parte entera y tipo de duración

  Parameters
  ---------
  df: object
    dataframe con columna duration
  """

  df['duration_int'] = df['duration'].apply(obtener_valor_tipo,args=(0,))

  df['duration_int'] = df['duration_int'].astype('float')
 
  df['duration_int'] = df['duration_int'].astype('Int64')
 
  df['duration_type'] = df['duration'].apply(obtener_valor_tipo,args=(1,))

  # eliminamos columna duration que ha sido separada y no es necesaria
  df.drop(columns='duration', axis=1, inplace=True, errors='ignore')

Agrupamos las funciones en una sola

In [8]:
def transformar(prefijo,df):
  """
  Aplica transformaciones al df

  Parameters
  ---------
  df: object
    dataframe a transformar
  prefijo: str
    letra que se añade al inicio de cada valor del campo show_id
  """

  generar_id(prefijo, df)

  completar_rating(df)    

  formatear_fecha(df)

  texto_en_minuscula(df)

  separar_duration(df)

  # reordenamos las columnas para que id este primer 
  columns = np.array(df.columns)

  columns = [column for column in columns if column != 'id']

  df = df[['id']+columns]

  return df

Transformamos los datasets

In [20]:
# la funcion obtener_datos_plataforma obtiene los datos crudos de google drive
df = obtener_datos_plataforma('amazon')

In [21]:
amazon_df = transformar('a',df)

In [22]:
df2 = obtener_datos_plataforma('disney')

In [23]:
disney_df = transformar('d',df2)

In [24]:
df3 = obtener_datos_plataforma('hulu')

In [25]:
hulu_df = transformar('h',df3)

In [26]:
df4 = obtener_datos_plataforma('netflix')

In [27]:
netflix_df = transformar('n',df4)

Exportamos las transformaciones

In [28]:
amazon_df.to_csv('../transformaciones/amazon.csv',index=False)

In [29]:
disney_df.to_csv('../transformaciones/disney.csv',index=False)

In [30]:
hulu_df.to_csv('../transformaciones/hulu.csv',index=False)

In [31]:
netflix_df.to_csv('../transformaciones/netflix.csv',index=False)